<a href="https://colab.research.google.com/github/laynz28/-NVC-/blob/main/RVC_MAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RVC-Project/Retrieval-based-Voice-Conversion-WebUI/blob/main/Retrieval_based_Voice_Conversion_WebUI_v2.ipynb) original notebook

In [ ]:
# @title #View graphics card
from IPython.display import clear_output
!nvidia-smi

In [ ]:
# @title #mount drive

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
#@title #Install dependencies (if you want)
!pip install pyngrok
!pip install tensorflow==2.12.0
!pip3 install faiss-cpu==1.7.2 fairseq gradio==3.14.0 ffmpeg ffmpeg-python praat-parselmouth pyworld numpy==1.23.5 numba==0.56.4 librosa==0.9.2
!python3 -m pip install -U demucs
!pip install yt_dlp
!pip install ffmpeg
clear_output()

In [ ]:
# @title #Clone repository
RVC = "https://github.com/HoshioPilio/project-main.git"

!git clone $RVC
%cd /content/project-main

In [ ]:

#@title #download files

Source = "download_files"

!python $Source.py

In [ ]:
#@title #Input Youtube WAV Download

Mode = "Separate"
dataset = "Youtube"
url = "https://youtu.be/Xu-69SCoPL4?si=kCJCKj1BvThPAgqj" #@param {type:"string"}
AUDIO_NAME = "idol" #@param {type:"string"}

In [ ]:
#@title #Download Youtube WAV
from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)

In [ ]:
#@title #Separate Vocal and Instrument/Noise using Demucs
import subprocess
AUDIO_INPUT = f"/content/youtubeaudio/{AUDIO_NAME}.wav"

if dataset == "Drive":
  command = f"demucs --two-stems=vocals {drive_path}"
elif dataset == "Youtube":
  command = f"demucs --two-stems=vocals {AUDIO_INPUT}"

result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())

In [ ]:

#@title #copy audio file to drive

!mkdir -p /content/drive/MyDrive/audio/{AUDIO_NAME}
!cp -r /content/separated/htdemucs/{AUDIO_NAME}/* /content/drive/MyDrive/audio/{AUDIO_NAME}
if dataset == "Youtube":
  !cp -r /content/youtubeaudio/{AUDIO_NAME}.wav /content/drive/MyDrive/audio/{AUDIO_NAME}
!mv /content/drive/MyDrive/audio/{AUDIO_NAME}/vocals.wav /content/project-main/audios

In [ ]:
# @title #run webui
ngrok_authtoken = "2VqrgGnzNs2bSC0racuiPMMq52U_2sW8K1uh5KTb7rkpNrT7H"#@param {type:"string"}
!ngrok config add-authtoken {ngrok_authtoken}
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!rm -r /content/sample_data
from pyngrok import ngrok
ngrok.kill()
print("Tensorboard NGROK URL:",end="")
ngrok_url = ngrok.connect(7860)
print(ngrok_url)
!python app.py --colab